# CS3630 Project 1: Trash Sorting Robot (Fall 2022)
## Brief
- Due: Tue, Sep 13 at 11:59pm on gradescope
- Hand-in: through Gradescope

## Getting started
In order to use a file as your own, once we give you the colab notebook link:
1. Click the “File” button on the toolbar at the top
2. Click “Save a Copy”
3. Work on the project within the copy (it will say “Copy of…”)

## Submission Instructions
In order to submit a file, once you complete the project:
1. Click the “File” button on the toolbar at the top
2. Click “Download,”
3. And then click “Download .ipynb”
4. You will now have the .ipynb file on your local machine.
5. Make sure it is named `project1.ipynb`
6. Submit the `project1.ipynb` file to gradescope


## Introduction
Welcome to your first project in CS3630 (Fall 2022)!

In this project, we will be building a (simulated) trash sorting robot as illustrated in the [textbook](http://www.roboticsbook.org/intro.html) for this course. In this scenario, the robot tries to sort trash of some pre-determined categories into corresponding bins. Please refer to [Chapter 2](http://www.roboticsbook.org/S20_sorter_intro.html) of the book for a more detailed description of the scenario. **This project is basically based on Chapter 2 of the textbook. Please use the same number and data in the textbook for each TODO.**

First, install gtsam and import some other useful libraries.

In [85]:
# To use on colab, run the following line
!pip install -U -q gtbook

In [86]:
#export
import gtsam
import numpy as np
import math
from enum import Enum
from gtbook.discrete import Variables

In [87]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
# Download the project1_test file to check your code on colab
! gdown --id 1m6K4c0njbAxHlBGM8FpDYYvdNDNdaxIA

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1m6K4c0njbAxHlBGM8FpDYYvdNDNdaxIA
To: /content/project1_test.py
100% 81.3k/81.3k [00:00<00:00, 67.8MB/s]


In [89]:
from project1_test import TestProject1
from project1_test import verify

In [90]:
np.random.seed(3630)
unit_test = TestProject1()

**IMPORTANT NOTE: Please use the variables provided for the results of each of the TODOs.**
## Modeling the World State ([Book Section 2.1](http://www.roboticsbook.org/S21_sorter_state.html))
- Functions to complete: **TODO 1**, **TODO 2**, and **TODO 3**
- Objective: Representing the prior probabilities of the trash categories and simulate it by sampling. Please use the prior probabilities provided in the textbook

In [91]:
#export
### ENUMS ###
#쓰레기의 종류
class Trash(Enum):
    CARDBOARD = 0
    PAPER = 1
    CAN = 2
    SCRAP_METAL = 3
    BOTTLE = 4

#버려야할곳
class Bin(Enum):
    GLASS_BIN = 0
    METAL_BIN = 1
    PAPER_BIN = 2
    NOP = 3

#쓰레기를 인식한 결과, 쓰레기의 타입
class Detection(Enum):
    BOTTLE = 0
    CARDBOARD = 1
    PAPER = 2


### CONSTANTS ###
# All possible trash categories
CATEGORIES = ['cardboard', 'paper', 'can', 'scrap_metal', 'bottle']

# All possible actions/bins (nop means no action)
ACTIONS = ['glass_bin', 'metal_bin', 'paper_bin', 'nop']


# Useful Global Variables
variables = Variables()
categories = CATEGORIES
#카테고리를 선언함..
Category = variables.discrete('Category', categories)

Conductivity = variables.binary('Conductivity')
# print(Conductivity) (1,2) 뭘까..그냥 2진수인듯..

Detection = variables.discrete('Detection', ['bottle', 'cardboard', 'paper'])

**TODO 1 & TODO 2**:

In [92]:
#export
# TODO 1:
# Prior probabilities
def get_category_prior():
    '''
    Returns the prior probabilities of the trash categories.

        Parameters:
            None

        Returns:
            category_prior (gtsam.DiscreteDistribution): a DiscreteDistribution
                that summarizes the prior probabilities of all trash categories
    '''
    category_prior = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    category_prior = gtsam.DiscreteDistribution(Category, "200/300/250/200/50")
    # pretty(category_prior)

    # index = categories.index('can')
    # Pcan = category_prior(index)
    # print(f"p can index: {Pcan}")

    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return category_prior
# get_category_prior()

# TODO 2:
# Prior probabilities PMF
def get_category_prior_pmf():
    '''
    Returns the probability mass function (PMF) of the prior probabilities
    of the trash categories.

        Parameters:
            None

        Returns:
            category_prior_pmf (list): a list of the PMF
    '''
    category_prior_pmf = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    
    category_prior = get_category_prior()
    category_prior_pmf = category_prior.pmf()
    # print(PMF)
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return category_prior_pmf



In [ ]:
print("Testing your prior probabilities of the trash categories: ")
# print(unit_test.test_get_category_prior_pmf, get_category_prior_pmf)
print(verify(unit_test.test_get_category_prior_pmf, get_category_prior_pmf))

Testing your prior probabilities of the trash categories: 
"Correct"


**TODO 3**:

In [93]:
#export
# TODO 3:
def sample_category():
    '''
    Returns a sample of trash category by sampling with the prior probabilities
    of the trash categories

        Parameters:
            None

        Returns:
            sample (int): an int indicating the sampled trash category, the
                int-category mapping is at the beginning of this notebook
    '''
    sample = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    CDF = np.cumsum(get_category_prior_pmf())
    # print(CDF)
    u = np.random.rand()
    #sample = cateogry in range five.
    #대충 random 값 u의 값이 CDF 어디 관할인지 리턴합니다.
    #예를 들어 u = 3.2 이면 CDF <=4 에 해당하므로 4번째 카테고리인 BOTTLE로 인식하려나
    #돌려봤는데 u = [0,1] 이고 뭐 그에 적절한 CDF 값이 분포되어있는듯..
    for sample in range(5):
        if u<float(CDF[sample]):
            # print(u,sample)
            return sample

    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    

In [ ]:
print("Testing your sample of trash category: ", verify(unit_test.test_sample_category, sample_category))

Testing your sample of trash category:  "Correct"


## Actions for Sorting Trash ([Book Section 2.2](http://www.roboticsbook.org/S22_sorter_actions.html))
- Functions to complete: **TODO 4**
- Objective: Representing actions and their corresponding costs, please use the data provided in the textbook

In [94]:
#export
# TODO 4:
'''
    Fill out the cost table with corresponding costs, where the rows correspond
    to ACTIONS and the columns correspond to CATEGORIES.
'''
COST_TABLE = None
###############################################################################
#                             START OF YOUR CODE                              #
###############################################################################

# categories = ["cardboard", "paper", "can", "scrap metal", "bottle"]
# actions = ["glass bin", "metal bin", "paper bin", "nop"]
#row: nth 쓰레기통에 버렸을 때에 cost 모음
#col:  쓰레기 모음
COST_TABLE = np.array([[2,  2,  4,  6,  0],
                 [1,  1,  0,  0,  2],
                 [0,  0,  5, 10,  3],
                 [1,  1,  1,  1,  1]])
###############################################################################
#                              END OF YOUR CODE                               #
###############################################################################

## Sensors for Sorting Trash ([Book Section 2.3](http://www.roboticsbook.org/S23_sorter_sensing.html))
- Functions to complete: **TODO 5-7** , **TODO 8-10** 
- Objective: Representing conditional probabilities of sensors and simulate them by sampling, please use the data provided in the textbook

**TODO 5-8**:

In [95]:
#export
# TODO 5:
# 1. Conductivity - binary sensor
def get_pCT():
    '''
    Returns P(Conductivity | Trash Category) = p(c,t)/p(t)

        Parameters:
            None

        Returns:
            pCT (gtsam.DiscreteConditional): a DiscreteConditional that
                indicates the conditinal probability of conductivity given
                the trash category
    '''
    pCT = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    pCT = gtsam.DiscreteConditional(
    Conductivity, [Category], "99/1 99/1 10/90 15/85 95/5")
    # print(pCT)
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return pCT

# TODO 6:
# 2. Detection - multi-valued sensor
def get_pDT():
    '''
    Returns P(Detection | Trash Category)

        Parameters:
            None

        Returns:
            pDT (gtsam.DiscreteConditional): a DiscreteConditional that
                indicates the conditinal probability of camera detection
                given the trash category
    '''
    pDT = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    pDT = gtsam.DiscreteConditional(
    Detection, [Category], "2/88/10 2/20/78 33/33/34 33/33/34 95/2/3")

    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return pDT

# TODO 7:
# 3. Weight - continuous-valued sensor
def get_pWT():
    '''
    Returns P(Weight | Trash Category)

        Parameters:
            None

        Returns:
            pWT (np.array): a numpy array of lists that consists of the means
                and standard deviations that define the weight distribution of each
                trash category

    '''
    pWT = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    pWT = np.array([[20, 10], [5, 5], [15, 5], [150, 100], [300, 200]])
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return pWT

# TODO 8:
def sample_conductivity(category=None):
    '''
    Returns a sample of conductivity using the conditional probability
    given the trash category.

        Parameters:
            category (int): an int indicating the trash category

        Returns:
            conductivity (int): an int indicating the conductivity, with
                0 being nonconductive and 1 being conductive
    '''
    conductivity = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    # values = gtsam.DiscreteValues()
    # print("1:",values)  DiscreteValues{}
    # values[Category[0]] = category
    # print("2:",values) DiscreteValues{0: 0}
    # print(get_pCT())
    #어짜피 pCT중에서 category 하나로 조회해보기때문에.. values로 list를 안만들어도 되지않을까..  
    conductivity = get_pCT().sample(category) 

    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return conductivity
  

In [ ]:
print("Testing your sample conductivity: ", verify(unit_test.test_sample_conductivity, sample_conductivity))

Testing your sample conductivity:  "Correct"


**TODO 9**:

In [96]:
#export
# TODO 9:
def sample_detection(category=None):
    '''
    Returns a sample of detection using the conditional probability given
    the trash category.

        Parameters:
            category (int): an int indicating the trash category

        Returns:
            detection (int): an int indicating the sampled detection, the
                int-detection mapping is at the beginning of this notebook
    '''
    detection = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    detection = get_pDT().sample(category)
    # print(detection)
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return detection

In [ ]:
print("Testing your sample detection: ", verify(unit_test.test_sample_detection, sample_detection))

Testing your sample detection:  "Correct"


**TODO 10**:

In [97]:
#export
# TODO 10:
def sample_weight(category=None):
    '''
    Returns a sample of weight using the conditional probability given
    the trash category.

        Parameters:
            category (int): an int indicating the trash category

        Returns:
            weight (double): a double indicating the sampled weight
    '''
    weight = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    # print(get_pWT()[category]) [20, 10]이 많네..
    #python에서 *을 앞에 붙이는 이유는 get_PWT()[category]의 값이 몇개일지 몰라서..
    #가우시안 분포를 사용하여 샘플을 추출하나보다..
    weight = np.random.normal(*get_pWT()[category])
    # print(weight)
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return weight

In [98]:
#export
# TODO 11:
def likelihood_no_sensors():
    '''
    Returns the likelihoods of all trash categories using only priors,
    aka no sensors.

        Parameters:
            None

        Returns:
            likelihoods (list): a list of likelihoods of each trash category
    '''
    likelihoods = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    likelihoods = []
    # print(get_category_prior().likelihood(0).enumerate()[0][1])
    # help(gtsam.DecisionTreeFactor)
    
    for i in range(5):
      #help에서 categroy_prior()의 본체(gtsam.DiscreteDistribution)인거 찾아보면 likelihood가나와..
      #보면 int값을 받는다는데 그냥 index값이야..
      #근데그게 list로 되어있어서 enumerate로 풀어야하는데,[0]로 받고 풀면 tuple이라 [1]에있는값이 float값임
      likelihoods.append(get_category_prior().likelihood(i).enumerate()[0][1])
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return likelihoods
# likelihood_no_sensors()

In [68]:
print("Testing your sample weight: ", verify(unit_test.test_sample_weight, sample_weight))

Testing your sample weight:  "Correct"


## Perception ([Book Section 2.4](http://www.roboticsbook.org/S24_sorter_perception.html))
- Functions to complete: **TODO 11-15** 
- Objective: Calculating likelihoods using different methods given the observations from the world, please use the data provided in the textbook

**TODO 11**:

In [99]:
print("Testing your likelihoods with no sensors: ")

print(verify(unit_test.test_likelihood_no_sensor, likelihood_no_sensors))

Testing your likelihoods with no sensors: 
"Correct"


Helper function you can use in the following TODOs

In [70]:
#export
### HELPER FUNCTIONS ###
def Gaussian(x, mu=0.0, sigma=1.0):
    return np.exp(-0.5 * (x - mu) ** 2 / sigma ** 2) / np.sqrt(2 * np.pi * sigma ** 2)

**TODO 12**:

In [100]:
#export
# TODO 12:
def likelihood_given_weight(weight):
    '''
    Returns the likelihoods of all trash categories using only the weight
    sensor (no priors)

        Parameters:
            weight (double): a double indicating the weight of trash

        Returns:
            likelihoods (list): a list of likelihoods of each trash category
    '''
    likelihoods = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    likelihoods =  np.array([Gaussian(weight, *get_pWT()[index]) for index in range(5)])
    # print(likelihoods)
    # print(weight)
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return likelihoods
likelihood_given_weight(30.430411912035503)

array([2.31562377e-02, 1.92630220e-07, 6.82103821e-04, 1.95189801e-03,
       8.04247955e-04])

In [72]:
print("Testing your likelihoods using only the weight sensor: ")
print(verify(unit_test.test_likelihood_given_weight, likelihood_given_weight))

Testing your likelihoods using only the weight sensor: 
"Correct"


**TODO 13**:

In [101]:
#export
# TODO 13:
def likelihood_given_detection(detection):
    '''
    Returns the likelihoods of all trash categories using only the detection
    sensor (no priors)

        Parameters:
            detection (int): an int indicating the sampled detection, the
                int-detection mapping is at the beginning of this notebook

        Returns:
            likelihoods (list): a list of likelihoods of each trash category
    '''
    likelihoods = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    likelihoods= []
    pDT = get_pDT() 
 
    #detection은 어떤 bin으로 감지되었는지를 나타낸다
    #likelihoods이므로 예컨데 bottle bin에 감지된 cardboard, paper, can, metal,bottle의 확률을 담고있다.
    #range의 수를 어떻게 알 수 있을까? 그것은 숙제다..
    for j in range(5):
      likelihoods.append(pDT.likelihood(detection).enumerate()[j][1])

    # print(likelihoods)
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return likelihoods
print(likelihood_given_detection(0))

[0.02, 0.02, 0.33, 0.33, 0.95]


In [74]:
print("Testing your likelihoods using only the detection sensor: ")
print(verify(unit_test.test_likelihood_given_detection, likelihood_given_detection))

Testing your likelihoods using only the detection sensor: 
"Correct"


**TODO 14**:

In [102]:
#export
# TODO 14:
def bayes_given_weight(weight):
    '''
    Returns the posteriors of all trash categories by combining the weight
    sensor and the priors

        Parameters:
            weight (double): a double indicating the weight of the trash

        Returns:
            posteriors (list): a list of posterior probabilities of each trash category
    '''
    posteriors = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    posteriors = []
    
    prior =get_category_prior_pmf()
   
    def normalized(w):
      Lx = likelihood_given_weight(w) *prior
      # sum = 0
      # for i in range(len(Lx)):
      #   print(Lx[i])
      #   sum+= Lx[i]
      # print("sum:",sum) !=1
      return Lx/np.sum(Lx)
    # for i in range(len(pcp)):
    #   prior.append(pcp[1])
    # print(prior)
    # print(likelihood_given_weight(weight))
    # print(likelihood_given_weight(weight),"\n", prior)
    # posteriors = (likelihood_given_weight(weight) * prior)/np.sum(likelihood_given_weight(weight)*prior)
    posteriors = normalized(weight)
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return posteriors
print(bayes_given_weight(30.430411912035503))
print( [0.885105674130704, 1.1044417252599398e-05, 0.03259024476725486, 0.07460780259602869, 0.00768523408875979])

[8.85105674e-01 1.10444173e-05 3.25902448e-02 7.46078026e-02
 7.68523409e-03]
[0.885105674130704, 1.1044417252599398e-05, 0.03259024476725486, 0.07460780259602869, 0.00768523408875979]


In [76]:
print("Testing your posteriors with the weight sensor and priors: ")
print(verify(unit_test.test_bayes_given_weight, bayes_given_weight))

Testing your posteriors with the weight sensor and priors: 
"Correct"


In [103]:
#export
# TODO 15:
# Bayes with three sensors
def bayes_given_three_sensors(conductivity, detection, weight):
    '''
    Returns the posteriors of all trash categories by combining all three
    sensors and the priors

        Parameters:
            conductivity (int): an int indicating the conductivity, with
                0 being nonconductive and 1 being conductive

            detection (int): an int indicating the sampled detection, the
                int-detection mapping is at the beginning of this notebook

            weight (double): a double indicating the weight of the trash

        Returns:
            posteriors (list): a list of posterior probabilities of each trash category
    '''
    posteriors = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    
    def posterior(c,d,w):
      prior = get_category_prior()
      conductivity_factor = get_pCT().likelihood(c)
      detector_factor = get_pDT().likelihood(d)
      weight_factor = gtsam.DecisionTreeFactor(Category, likelihood_given_weight(w))
      Px = gtsam.DiscreteDistribution(conductivity_factor* detector_factor * weight_factor * prior)
      return Px.pmf()
    posteriors = posterior(conductivity,detection,weight)
    
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return posteriors
print("result:",bayes_given_three_sensors(0 ,2 ,30.430411912035503))
print("answer:",[0.944585093914191, 9.19355273149246e-05, 0.011944756687272868, 0.04101712285436969, 0.002361091016851618])

result: [0.944585093914191, 9.19355273149246e-05, 0.011944756687272868, 0.04101712285436969, 0.002361091016851618]
answer: [0.944585093914191, 9.19355273149246e-05, 0.011944756687272868, 0.04101712285436969, 0.002361091016851618]


**TODO 15**

In [78]:
print("Testing your posteriors giving all three sensors: ")
print(verify(unit_test.test_bayes_given_three_sensors, bayes_given_three_sensors))

Testing your posteriors giving all three sensors: 
"Correct"


## Decision Theory ([Book Section 2.5](http://www.roboticsbook.org/S25_sorter_decision_theory.html))
- Functions to complete: **TODO 16** 
- Objective: Incorporating the cost table with the perception to reach a final sorting decision

**TODO 16**:

In [104]:
#export
# TODO 16:
### DECISION ###
def make_decision(posteriors):
    '''
    Returns the decision made by the robot given the likelihoods/posteriors you calculated

        Parameters:
            posteriors (list): a list of posteriors of each trash category

        Returns:
            action (int): an int indicating the action taken by the robot, the
                int-action mapping is at the beginning of this notebook
    '''
    action = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    
    expected_cost= COST_TABLE @ posteriors
    action = np.argmin(expected_cost)

    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    return action

In [80]:
print("Testing the decision made by your robot: ")
print(verify(unit_test.test_make_decision, make_decision))

Testing the decision made by your robot: 
"Correct"


In [105]:
unit_test.get_cost_table(COST_TABLE)
print("Testing your cost without sensors: ")
print(verify(unit_test.test_score_likelihood_no_sensor, likelihood_no_sensors, make_decision))
print("Testing your cost using the weight sensor:")
print(verify(unit_test.test_score_likelihood_given_weight, likelihood_given_weight, make_decision))
print("Testing your cost using the detection sensor:")
print(verify(unit_test.test_score_likelihood_given_detection, likelihood_given_detection, make_decision))
print("Testing your cost using with the weight sensor and priors:")
print(verify(unit_test.test_score_bayes_given_weight, bayes_given_weight, make_decision))
print("Testing your cost using all three sensors: ")
print(verify(unit_test.test_score_bayes_given_three_sensors, bayes_given_three_sensors, make_decision))

Testing your cost without sensors: 
"Correct"
Testing your cost using the weight sensor:
"Correct"
Testing your cost using the detection sensor:
"Correct"
Testing your cost using with the weight sensor and priors:
"Correct"
Testing your cost using all three sensors: 
"Correct"


## Extra Credit: Learning ([Book Section 2.6](http://www.roboticsbook.org/S26_sorter_learning.html))
A Gaussian distribution, also known as a normal distribution, is an inappropriate distribution to represent
the weight of an item. This is because it has an infinite range and therefore sampling from it can produce
a negative number, while an item cannot have a negative weight. A more commonly used distribution
used to represent weight is the [log-normal distribution](https://en.wikipedia.org/wiki/Log-normal_distribution) which can only contain positive real values. The book explains how to fit a gaussian distribution to a set of data. For extra credit, we would like you to implement a function. 
- Functions to complete: **TODO 17** 
- Objective: Fit a Log-Normal Distribution to a set of data
- Hint: There is an estimation of parameters section on the wikipedia article

**TODO 17**:

In [144]:
#export
# TODO 17
def fit_log_normal(data):
    '''
    Returns mu, sigma for a log-normal distribution

        Parameters:
            data (list of floats): A list of positive floats that represent the weight of an item

        Returns:
            mu (float), sigma (float): The mu and sigma for a log-normal distribution
    '''
    mu = None
    sigma = None
    ###############################################################################
    #                             START OF YOUR CODE                              #
    ###############################################################################
    mu = 0
    sigma = 0
    for i in range(len(data)):
      mu += np.log(data[i])
    mu = mu/len(data)

    for i in range(len(data)):
      sigma+= (np.log(data[i])- mu)**2
    sigma = np.sqrt(sigma/len(data))
    ###############################################################################
    #                              END OF YOUR CODE                               #
    ###############################################################################
    # correct = [(1., 0.5), (15., 2.5), (40., 1)]
    return mu, sigma
fit_log_normal([34959170.79815449, 51762359.567799196, 47411415.872367196, 2323061.2052304405, 69136.60352614753, 135205324.4963425, 2112793.5100784358, 162120.4035102557, 174838.98090694202, 3645862.721465153, 13594815.329121217, 3913192.050610969, 9415029.573954212, 21500451.1624513, 862443.5077040627, 7975624.636630565, 39434.50056295492, 34158147.827099964, 402809.79280133225, 218067.08998712068, 60411725.994309, 605241888.1164209, 1363275.0739148378, 10772144.284432009, 1857327.674815533, 992282.5553522258, 699130.0595396446, 84357250.38940556, 32684945.89935913, 585137949.7162265, 22816.496057882665, 1391251.4327272165, 5710464.671517986, 3510382.9348121695, 31944288.508223437, 100729890.84866926, 447586.32335878734, 16347041.248221586, 1236911.538456917, 89391459.54134256, 85350376.41129899, 39715437.00142654, 12391764.883322308, 651862.9582796209, 251991.42832592892, 62255510.83612925, 1007075.4409851624, 607701.5509874969, 2635597.763495296, 4840002.026798173, 6245856.437278226, 78621843.94511339, 36086258.59324927, 2685212.61316618, 1147911.7914833932, 213825.4964948427, 207505576.33175737, 663169.9406283341, 371849.22285000485, 2826359.837514205, 3979329.5233983435, 15031505.74984141, 21811368.29896711, 24246.619995083434, 47521872.726806685, 911319.1322961944, 89971.77068326161, 474387.7559174091, 2378519.92244981, 3797435.7244466767, 15195139.489917442, 6180152.983316545, 40363557.25836232, 5190054.289988422, 2029.2514931732333, 22417.84272039614, 648006.2639702444, 10337080.513209049, 865247.6109671893, 9426538.47201721, 344388.4671750818, 13941039.837901939, 513300.5519063588, 4849186.332673296, 1040477345.1689602, 1126176.5816288188, 369881.47696221573, 988249.7678540999, 3780632.027657672, 19903516.16150616, 217828234.48609757, 278523.6869486152, 2306420.972434129, 4872233.33180074, 31435237.74765993, 2467805.497765867, 288112752.67221916, 1503489.8566503595, 4826499.113374603, 15178646.30026637, 3642935.5642547016, 2441452.057395124, 143655780.48211208, 493791.5086104588, 50508719.096470274, 742295.9951673608, 121378.27710736195, 1932105.4565313498, 452213.7208005129, 11325845.50685493, 425043.95373729686, 216256.41874249288, 15834117.405976877, 15745388.998904485, 13707079.360598, 2185025.41348989, 32372724.95155905, 69267112.21776845, 5375648.703873337, 45028.17493743917, 3410972689.1480103, 1148712.9657462244, 10902120.893497251, 359171.53925915103, 3026545.1504878704, 57174956.47872378, 89202.40277911433, 102306.11533596525, 2195411.9907988803, 984171.4988037598, 22649202.507776875, 102961112.65430246, 1675923.463536121, 14274130.89362232, 404773.5067981565, 5211030.48989417, 9947196.546449076, 43341669.831391655, 2171618.330945271, 7433044.430020651, 260698.78680298937, 22453567.076657757, 66933254.54997351, 3158.9652299450026, 382990.6669832005, 1018533975.4352785, 621180.0490795928, 3737070.3730835705, 19470658.326553706, 1131348.4934714115, 2882852.801039179, 121903.24605072157, 3720398.18153823, 27468397.123036504, 8591125.489156036, 7752892.037358855, 22187632.557473183, 6297988.999190813, 95458467.10606267, 1763187.376165131, 9864843.631960746, 2277190.945225595, 1315855.6458266368, 6839208.749116727, 1756785.9216212798, 1488344.0672796841, 9424165.770354828, 164518923.39816815, 669599.0165744931, 23193886.461567067, 633653.7966263836, 238259.62974306213, 16122207.216432795, 82175.10640711828, 101303.89182977572, 739664.9680500972, 10881418.484422611, 32939431.066386063, 47975.85336744716, 277336.7985905164, 127155.77893336014, 5647924.053421697, 1951845036.9992006, 56926083.25227334, 24285785.452991717, 25803046.635584213, 2942959.286756438, 57992.35365928826, 12576164.082402091, 51179481.81765074, 2206536.873583655, 25494956.736195996, 1935731.3638287513, 3502278.0379490824, 6890875.587762226, 994603.3270293443, 32740926.470288813, 2202826.9996845075, 42687.91847284504, 16968551.88637815, 665564966.5213491, 9413985.903730037, 38154472.90636372, 30215.133398358346, 7402613.746510946, 18155085.57512425, 898856.0474914699, 331668.6933812439, 108804115.79126394, 1812762.404097657, 255253664.75574464, 3014146.3724788968, 28751.435675587476, 31208185.87495017, 1026266.8815786259, 89246205.13221066, 17262638.831092812, 1865381.9342335304, 1748215.7569946605, 124034.75025654804, 41619202.66538399, 5588371.804682868, 20115195.877021976, 8553558.468288718, 50121.63260415051, 985750.1997994779, 166523953.97175816, 5018830.575130827, 51188902.47655981, 26093900.09649721, 1427501.2997749348, 17716284.072748713, 183426.39787884243, 16469211.731848612, 11470080.455519762, 11613702.577477362, 10507604.085691074, 1110717.919996887, 175816.65879418747, 260625.84564502005, 49597264.04672047, 1226102.195184747, 764101.3797972897, 257518.9969100542, 7156371.845404919, 651676702.6774925, 686821.6527565417, 57970.11657538711, 28961775.339314725, 1321355.650682047, 35755.03747846936, 22182093.19654498, 7884640.592397745, 117906599.52878442, 9873666.6449838, 151885.90542066086, 2233.9096224185178, 442866.09445200075, 49245550.52667468, 120650.33357555153, 25186798.35464349, 62460217.95039092, 707996.4333462089, 361346138.6369022, 11594910.661597375, 3454251.4825129635, 1597663.7684472443, 863351.1477660816, 397941.310189435, 280456.4559761425, 3327178.0530476593, 1759895.120997173, 403582.3113378798, 401141.60266471305, 6759145.567912643, 15519761.092734206, 14518695.970883856, 16175424.771991238, 17010110.27931273, 15695428.238936769, 147349.82668640357, 1339215.5917345595, 345860399.3950257, 2982128.1704469407, 2361365.0841752654, 184064.56753235662, 7733436.461960154, 1497480.162647485, 664918.5167209067, 134162.77314981713, 443795.61785385903, 16397194.040671093, 5191581.923486834, 1575136.2842830988, 302695402.5530284, 5998334.550781204, 854650.6877557691, 1269922.1641711667, 1099662.0848208219, 9276124.831752568, 87859525.14421321, 2739727.7677268316, 101902.96507766304, 3993355.161926736, 1873129264.7852714, 307950.5081551801, 9942946.82318662, 45820272.11870012, 217054.6808536032, 835089.0135882524, 900519.7467614461, 4198805.14306867, 8347207.930507413, 467816.96760509396, 3781936.3537193714, 9121784.411686461, 120001855.32790826, 61411.18726970955, 9506745.00379699, 108799752.95801271, 3446682.5005133157, 2557502.2959999447, 18671814.92196235, 12043051.9831913, 14819.704119945645, 115016.65468144295, 14694318.862984806, 282371.2234551786, 11289670.133766335, 1014999.8511522505, 3899514.9836885645, 111230936.19313078, 10774904.04418183, 682765.7519696663, 12900827.0630308, 134298.5848606557, 228686.35622919802, 2488977.008257531, 3921553.6046407847, 3788717.359125665, 137336.59184470342, 15015480.008744888, 28285.19588240316, 7682654.005142792, 2647339.4783080127, 1786937.9087369137, 31112190.638286613, 327334.63807286636, 25227183.498287253, 329657819.47951734, 3558547.2795702475, 596338372.5859632, 53186235.23214591, 2638350.7693080544, 10254006.4687673, 3145493181.6339273, 263998.93533773755, 2433556.148959878, 68884930.80934235, 149536.54977613554, 2243386.658906257, 10088984.01542491, 12468832.416438172, 14253377.953919593, 15835068.309648497, 715570.0740787503, 577177588.1956298, 3992695.8121820544, 7088876.562160069, 162585.77120945556, 496810.6430653547, 46853656.591053456, 669234.7637317437, 64134.88153309388, 177482240.30000207, 734826.729175232, 137365645.56012264, 175371227.06497565, 6962190.1372932, 1762740.3042732782, 64936500.96318961, 5431960.095131799, 142806136.86748743, 566965.8135687213, 671041.2762788621, 1526390.820647165, 197370.12680284033, 14367.56927319693, 476549.81951030076, 18704.615738810775, 25288278.30067324, 740450.4119420223, 6096215.834177068, 63932660.10946646, 185090795.04754665, 134816634.11874816, 2388354.703103274, 390772.9393102029, 51014975.01936643, 44958380.53701657, 5818356.469272307, 917035.3985140527, 17342.128999268996, 20191.947977757703, 286940086.768623, 15614539.579929963, 14550435.65131192, 3292098.4390004636, 736886.0529954383, 12251.829531158945, 2282012.5377453803, 1772082.8675332326, 2479292.088204756, 807368.656599975, 119784.41863329038, 1045730.2599052908, 4801446.385065489, 703161.6174592598, 810247.3145436104, 87866.76969572212, 63764645.64236932, 972616.5148920766, 26894666.01589259, 2531775.6421332783, 38077659.73853053, 32426225.0713373, 2013713.170335323, 18949995.846282054, 1687433.2321010043, 490224.5623409384, 8357217.608505889, 8787750.609620811, 358973.0406967133, 1068023.2812609766, 5142573.798415881, 47912.46676055445, 15297392.999070004, 1351802.251104918, 7958332.262722489, 30612.402732353, 3583629.696498614, 117171.86055557516, 389665933.2518792, 3167650.9542215196, 21271102.243357494, 11237394.260670332, 10921992.679816801, 127941004.28064612, 172343.1883634066, 231584.94670652554, 1144024.2275092823, 1716571.15299332, 74196329.96084905, 203394.3896524597, 875536970.4276268, 3220514.238135394, 4142555.2488185866, 19449883.357014265, 5168041.884044745, 30116180.308606125, 38385151.74425829, 6032561.444625929, 1125000.7228747664, 15151417.09629641, 91299.33861005989, 8721016.589194106, 1277391.7278565892, 3503721.507627974, 1258587.4491948094, 8253999.525692791, 1276642.8019395173, 211947.09889181601, 8098109.847646128, 6941849.545742968, 377950.2596515228, 9333814.935561808, 2957176.632299567, 173036.3434170908, 1277948.1879974848, 419963.5394936271, 1661158.184012997, 230478.05924625788, 21955416.83842553, 210417.84360672528, 437337.27131483186, 35511883.39958869, 822955.6142607032, 5184585.818857267, 12330411.523710733, 671973.0020299773, 13929066.91191024, 3844579.9554627123, 7868437.1805505995, 287498600.0072972, 5636386.864377269, 46556150.23477381, 89623.71014970404, 3088232.954783563, 1214649.6083924847, 5893963.454536498])

(15.086005819289724, 2.441810546858024)

In [145]:
print("Testing your log-normal distribution: ", verify(unit_test.test_fit_log_normal, fit_log_normal))

Testing your log-normal distribution:  "Correct"
